# Suivi de l'apprentissage dans KERAS: illustration de TensorBoard pour suivre en temps réel l'apprentissage de modèles 

Dans ce notebook nous allons illustrer l'utilisation de **TensorBoard** pour suivre en temps réel l'apprentissage de modèles avec Keras.

Pour cela nous travaillerons sur le même jeu de données que celui du TP principal, le jeu MNIST, et nous utiliserons un modèle de régression multinomiale.

Le plan sera le suivant : 
* nous commencerons par mettre en forme le jeu de données et reproduire l'expérience du TP précédent
* nous verrons ensuite comment introduire un callback tensorboard et comment visualiser les résultats obtenus
* nous illustrerons comment cela peut-être utile pour mesurer l'impact du "learning rate" de l'algorithme de descente de gradient "classique"
* enfin, nous généraliserons ceci pour mesurer l'impact de l'algorithme d'optimisation, en considérant l'algorithme "Adam"


**Précision importante** : ce notebook n'est valable que pour **tensorflow v2**. La plupart des commandes devraient fonctionner pour une version plus ancienne de keras / tensorflow, mais les imports seront à revoir...

# 1) Mise en forme du jeu de données

## On commence par importer les packages & fonctions nécessaires

In [ ]:
# import packages
import csv
import os
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist
from tensorflow.keras.callbacks import TensorBoard

# set random seed
np.random.seed(123)

In [ ]:
%load_ext tensorboard

## On met ensuite en forme le jeu de données.

Pour cela : 
* on le charge via la fonction disponible dans Keras
* on ramène l'intensité des images dans la gamme \[0,1\]
* on convertit les images 28x28 en vecteurs (de longueur 84).
* on transforme les labels au format "one-hot-encoding"

In [ ]:
# load data
(X_train, y_train), (X_test, y_test) = mnist.load_data()
print('size of training data: ')
print(X_train.shape)
print('size of test data: ')
print(X_test.shape)

# convert our data type to float32 and normalize our data values to the range [0, 1]
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

# cast to vector
input_dim = 28*28 
X_train_vec = X_train.reshape(60000, input_dim) 
X_test_vec  = X_test.reshape(10000, input_dim)

# cast outcome to one-hot-encoding
Y_train = to_categorical(y_train)
Y_test  = to_categorical(y_test)

 On sépare enfin les données d'apprentissage en données d'apprentissage et de validation

In [ ]:
from sklearn.model_selection import train_test_split
X_train_vec_train, X_train_vec_val, Y_train_train, Y_train_val = train_test_split(X_train_vec, Y_train, test_size=0.2, random_state=42)

## On définit un modèle de régression multinomiale

In [ ]:
mlp = Sequential()
mlp.add(Dense(10, activation='softmax',input_dim=input_dim))

## On apprend notre modèle avec un algorithme SGD "standard", et ses paramètres par défaut.

In [ ]:
# compile with standard sgd
mlp.compile(optimizer='sgd', loss='binary_crossentropy', metrics='accuracy')
# fit
mlp.fit(X_train_vec_train, Y_train_train, validation_data = (X_train_vec_val,Y_train_val), batch_size=32, epochs=10, verbose=1) 

# 2) Introduction d'un "callback" Tensorboard

## On apprend notre modèle en introduisant un "callback" Tensorboard

Pour cela, il suffit d'appeler la fonction Keras correspondante, et de spécifier un chemin vers un dossier.

In [ ]:
# create model
mlp = Sequential()
mlp.add(Dense(10, activation='softmax',input_dim=input_dim))
# compile with standard sgd
mlp.compile(optimizer='sgd', loss='binary_crossentropy', metrics='accuracy')

# define tensorboard call-back
callback_tensorboard = TensorBoard(log_dir = "./Illustration-tensorboard/mlp_sgd-default", histogram_freq = 1)
# create output dir
if not os.path.exists("./Illustration-tensorboard"):
    os.mkdir("./Illustration-tensorboard")

# fit
mlp.fit(X_train_vec_train, Y_train_train, validation_data = (X_train_vec_val,Y_train_val), 
        callbacks = callback_tensorboard, batch_size=32, epochs=10, verbose=1) 

## Une fois qu'on a lancé l'apprentissage, on peut suivre l'évolution des performances (en temps réel ou a posteriori) en lançant l'outil TensorBoard.

Cela peut se faire directement au sein du notebook via la commande ci-dessous, mais il est également possible de lancer cette commande dans un terminal. Tensorboard vous renverra alors une URL à ouvrir dans un navigateur internet. 

Cette deuxième solution est souvent la plus utile, car elle permet de suivre l'apprentissage d'un modèle lancé via un script python classique (et non au sein d'un notebook comme ici), et même de suivre dans une même interface plusieurs apprentissages lancés en parallèle.

**A noter que la commande tensorboard prend un dossier en argument**. Il s'agit du dossier dans lequel sont écrites les sorties des différents "callbacks" que l'on veut suivre. Chaque modèle que vous souhaitez suivre dans cette instance tensorboard doit écrire dans son propre sous-dossier (comme ci-dessus, dans le dosser *./Illustration-tensorboard/mlp_sgd-default*)


In [ ]:
%tensorboard --logdir ./Illustration-tensorboard

# 3) Apprentissage de plusieurs modèles et comparaison via Tensorboard

## On apprend maintenant plusieurs modèles, en faisant varier le paramètre de "learning rate" et en introduisant un callback tensorboard à chaque fois.

Comme mentionné précédemment, il faut prendre soin de choisir un nouveau sous-dossier au sein du dossier "suivi" par Tensorboard pour chaque  modèle que l'on souhaite suivre. 

In [ ]:
##################################
# assess effect of learning rate #
##################################
learning_rates = [0.0001,0.001,0.01,0.1,0.5]
from tensorflow.keras.optimizers import SGD
for eta in learning_rates:
    # create model
    mlp = Sequential()
    mlp.add(Dense(10, activation='softmax',input_dim=input_dim))
    # compile with standard sgd
    mlp.compile(optimizer=SGD(lr=eta), loss='binary_crossentropy', metrics='accuracy')
    # define tensorboard call-back
    callback_tensorboard = TensorBoard(log_dir = "./Illustration-tensorboard/mlp_sgd-eta-{}".format(eta), histogram_freq = 1)
    # fit
    mlp.fit(X_train_vec_train, Y_train_train, validation_data = (X_train_vec_val,Y_train_val), 
            callbacks = callback_tensorboard, batch_size=32, epochs=10, verbose=1) 


## L'intérêt de tensorboard prend alors tout son sens : il est maintenant très facile d'explorer l'impact des hyper-paramètres sur l'apprentissage du modèle.

# 4) Analyse de l'impact de l'algorithme de descente de gradient 

## <span style="color:red"> EXERCICE : faire de même pour évaluer l'impact de l'algorithme d'optimisation à proprement parler, en considérant cette fois l'algorithme "Adam" </span>

Pour cela, il suffit d'importer et d'utiliser l'"optimizer" **Adam** comme nous nous l'avons fait ci-dessus pour le **SGD**.
Comme précédemment, nous ferons varier le paramètre de learning rate pour mesurer l'impact de la combinaison \[algorithme x learning-rate\]. 


In [ ]:
#######################################################################
#  TODO : reproduce previous experiment to assess effect of optimizer #
######################################################################
from tensorflow.keras.optimizers import Adam

